In [5]:
import pandas as pd
import numpy as np
import sklearn
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
raw_train = pd.read_csv('data/train.csv')
print raw_train.shape
print raw_train.head()

(1460, 81)
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2 

In [30]:
# get columns with too many (>10%) null values
nas = raw_train.isnull().sum()
nas_nulls = nas[nas>raw_train.shape[0]/10]
print "columns to be removed\n",nas_nulls
nancols = nas_nulls.index.tolist()

columns to be removed
LotFrontage     259
Alley          1369
FireplaceQu     690
PoolQC         1453
Fence          1179
MiscFeature    1406
dtype: int64


In [33]:
processed_data = raw_train.drop(set(nancols),1)

In [45]:
# first quickly pick out columns with integer types
types = processed_data.dtypes
ints = types[types == 'int64'].index.tolist()
numericals = processed_data[ints]

In [46]:
print numericals.head()

   Id  MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
0   1          60     8450            7            5       2003          2003   
1   2          20     9600            6            8       1976          1976   
2   3          60    11250            7            5       2001          2002   
3   4          70     9550            7            5       1915          1970   
4   5          60    14260            8            5       2000          2000   

   BsmtFinSF1  BsmtFinSF2  BsmtUnfSF    ...      WoodDeckSF  OpenPorchSF  \
0         706           0        150    ...               0           61   
1         978           0        284    ...             298            0   
2         486           0        434    ...               0           42   
3         216           0        540    ...               0           35   
4         655           0        490    ...             192           84   

   EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  Misc